In [9]:
import pandas as pd
import numpy as np
from random import sample
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import pickle

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/trainwithzerostopredict.csv')
topredict= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/topredict.csv')
df_withoutzero= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/trainwithoutzeros.csv')

***df has all the topredict values entered as 0 for the Book-Rating column.***</br>
***topredict has all the values that we want to predict.***</br>
***df_withoutzero has all the topredict all the values including the values in the test dataset.***



In [12]:
df['User-ID'].nunique()

77805

In [13]:
topredict['User-ID'].nunique()

19935

##**500 users that rate the most**

In [14]:
df_num_rating= df.groupby('User-ID').agg(Number_of_ratings=('Book-Rating','count')).reset_index()
# Num ratings count

In [15]:
df_num_rating.sort_values(by=['Number_of_ratings'], ascending=False, inplace=True)
df_num_rating

,User-ID,Number_of_ratings
3160,11676,8524
27626,98391,5802
43027,153662,1969
52924,189835,1906
6510,23902,1395
...,...,...
23601,84129,1
46266,165812,1
9606,34231,1
46268,165826,1


In [16]:
top_users= df_num_rating['User-ID'][:500]
# top 500 users

##**To Predict User List and Batch Designation**

In [17]:
topredict_users= topredict['User-ID'].unique()
topredict_users

array([    17,     56,    114, ..., 278844, 278851, 278854])

In [18]:
len(topredict_users)

19935

**We divide the test into 20 folds because of memory limitations. We will predict the test values by loading in batches**

In [19]:
dict_batch= {}
for i in range(0,len(topredict_users),1000):
  dict_batch['fold_{}'.format(i//1000)]= topredict_users[i:i+1000]

In [21]:
# this is the matrix factorization function from the MF notebook. 
def MF(M,k,max_it,lambd,mu):
    n=M.size()[0]
    m= M.size()[1]
    nonzero=len(M.nonzero())
    index= M.nonzero().split(1, dim=1)
    #param=torch.rand(n*k+k*m,dtype=float,requires_grad=True)
    param1=torch.rand((n,k),dtype=torch.float,requires_grad=True)
    param2=torch.rand((k,m),dtype=torch.float,requires_grad=True)

    opt1= torch.optim.Adam([param1],lr=0.1)
    opt2= torch.optim.Adam([param2],lr=0.1)
    
    #scheduler1= ReduceLROnPlateau(opt1, 'min') 
    #scheduler2 = ReduceLROnPlateau(opt2, 'min')
    

    #def get_loss(params,params_hat):
        #return torch.sum(torch.square(params- params_hat))

    def run_iterations(max_it):
        loss_record=[]
        converged=False
        for it in tqdm(range(max_it)):
            if it%2==0:
                opt1.zero_grad(set_to_none=True)
                #torch.matmul(param[:n*k].reshape(n,k), pam[n*k:].reshape(k,m))
                loss=torch.sum(torch.square(torch.matmul(param1, param2)[index]- M[index])) + lambd*torch.sum(torch.square(param1))+mu*torch.sum(torch.square(param2))
                loss_record.append(loss.item())
                loss.backward()
                opt1.step()
                #scheduler1.step(loss)
            else:
                opt2.zero_grad(set_to_none=True)
                #torch.matmul(param[:n*k].reshape(n,k), pam[n*k:].reshape(k,m))
                loss=torch.sum(torch.square(torch.matmul(param1, param2)[index]- M[index])) + lambd*torch.sum(torch.square(param1))+mu*torch.sum(torch.square(param2))
                loss_record.append(loss.item())
                loss.backward()
                opt2.step()
                #scheduler2.step(loss)
        display(loss_record)
        return torch.matmul(param1,param2)
    return run_iterations(max_it)

**The following loop loads the ith batch of the train dataset and constructs a matrix using the ith batch and the subset of the train dataset containing the 500 top users. Then it computes the matrix factorization prediction on the ith batch. It calculates the MSE for each batch in each iteration.**

In [ ]:
mse_batch=[] # this will collect the batch MSE
indices_batch=[] # this will collect the indices of the test dataset in each batch
for i in range(len(dict_batch)): #length of dict_batch is 20 because we have 20 folds
  df_batch= topredict[topredict['User-ID'].isin(dict_batch['fold_{}'.format(i)])].reset_index(drop=True)
  df_matrix= df[df['User-ID'].isin(list(dict_batch['fold_{}'.format(i)])+list(top_users))].reset_index(drop=True) #combines data from 500 top users and the test batch
  mat= df_matrix.pivot(index='User-ID',columns='ISBN',values='Book-Rating') .fillna(0) # matrix using pivot
  matrix= torch.tensor(mat.values)/10 #converts to tensor and scales the values by 1/10
  dict_user= dict(zip(sorted(set(df_matrix['User-ID'])),range(len(sorted(set(df_matrix['User-ID']))))))
  dict_book= dict(zip(sorted(set(df_matrix['ISBN'])),range(len(sorted(set(df_matrix['ISBN']))))))
  index1=[] #this will collect the row index
  index2=[] #this will collect the column index
  for j in range(len(df_batch)):
    index1.append([dict_user[df_batch['User-ID'][j]]])
    index2.append([dict_book[df_batch['ISBN'][j]]])
  indices_topred = (torch.tensor(index1),torch.tensor(index1)) #this contains the indices from the matrix whose values we want to predict
  # this is a bit peculiar but torch tensors work this way
  df_actual= df_withoutzero[df_withoutzero['User-ID'].isin(list(dict_batch['fold_{}'.format(i)])+list(top_users))]
  # df_actual is the matrix with the ratings of the test dataset included
  # we need this to calculate MSE
  actual_matrix= df_actual.pivot(index='User-ID',columns='ISBN',values='Book-Rating') .fillna(0)
  actual_matrix= torch.tensor(actual_matrix.values)/10 # convert to tensor and scale like above

  mse_batch.append(100*torch.sum(torch.square(MF(matrix,10,1500,0.1,0.1)[indices_topred]- actual_matrix[indices_topred])))
  # we multiply by 10^2 because we scaled by 1/10 and squared
  # the values 25,0.1,0.1 come from the MF validation notebook where we performed validation for parameter tuning 
  indices_batch.append(len(indices_topred[0]))
  display(mse_batch[-1])

In [23]:
with open("/content/drive/MyDrive/Colab Notebooks/mse_batch", "wb") as fp:   #Pickling
  pickle.dump(mse_batch, fp)

with open("/content/drive/MyDrive/Colab Notebooks/indices_num_batch", "wb") as fp:   #Pickling
  pickle.dump(indices_batch, fp)

In [34]:
mse= sum(mse_batch)/sum(indices_batch)

In [35]:
mse

tensor(7.8160, dtype=torch.float64, grad_fn=<DivBackward0>)

In [36]:
rmse= np.sqrt(mse.detach())
rmse

tensor(2.7957, dtype=torch.float64)